In [1]:
import os

directory = f'/Users/lilimatic/greenspace'
os.chdir(directory)

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

from sklearn.utils.class_weight import compute_sample_weight

In [2]:
from functions.ml_data import *

In [3]:
df_copy = df.copy()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df_copy.loc[:, df_copy.columns != 'covidsafety'], \
                                                    df_copy['covidsafety'], test_size=0.3)

In [5]:
sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=y_train #provide your own target name
)


In [6]:
model = XGBClassifier(n_estimators=5, max_depth=5, learning_rate=1e-4, objective='binary:logistic')


#Model fit 
model.fit(X_train, y_train)

# Prediction
preds = model.predict(X_test)
preds


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [7]:
#Accuracy
np.mean((preds-y_test)**2)

0.4583333333333333

In [8]:
#Training error 
# Prediction
preds = model.predict(X_train)
np.mean((preds-y_train)**2)

0.42857142857142855